In [4]:
import pandas as pd
from pymongo import MongoClient, GEO2D
import time
import csv
import json
import pymongo
from sshtunnel import SSHTunnelForwarder
from bson.son import SON

In [31]:
foodRatings = pd.read_csv('../data/foodRatings.csv',encoding='latin-1')
pharmacies = pd.read_csv('../data/Pharmacy.csv',encoding='latin-1')
stations = pd.read_csv('../data/Station use.csv')
crime2 = pd.read_csv('../data/police/2017-02-hampshire-street.csv')
crime3 = pd.read_csv('../data/police/2017-03-hampshire-street.csv')
crime4 = pd.read_csv('../data/police/2017-04-hampshire-street.csv')
crime5 = pd.read_csv('../data/police/2017-05-hampshire-street.csv')
crime6 = pd.read_csv('../data/police/2017-06-hampshire-street.csv')
crime7 = pd.read_csv('../data/police/2017-07-hampshire-street.csv')
crime8 = pd.read_csv('../data/police/2017-08-hampshire-street.csv')
crime9 = pd.read_csv('../data/police/2017-09-hampshire-street.csv')
crimes = pd.concat([crime2,crime3,crime4,crime5,crime6,crime7,crime8,crime9])
properties = pd.read_csv('../data/propertylisting.csv')

In [13]:
frames

[        {5BBE9CB3-6332-4EB0-9CD3-8737CEA4A65A}   42000  1995-12-21 00:00  \
 0       {20E2441A-0F16-49AB-97D4-8737E62A5D93}   95000  1995-03-03 00:00   
 1       {D893EE64-4464-44B5-B01B-8E62403ED83C}   74950  1995-10-03 00:00   
 2       {F9F753A8-E56A-4ECC-9927-8E626A471A92}   43500  1995-11-14 00:00   
 3       {E166398A-A19E-470E-BB5A-83B4C254CF6D}   63000  1995-09-08 00:00   
 4       {A9A3C463-3CA0-4D71-8CF9-83B4D0536EEA}   29995  1995-05-12 00:00   
 5       {8DE57D57-DDC3-4993-8C33-83B4E5BAA543}  105000  1995-11-30 00:00   
 6       {568DE11A-432B-4DA3-B9A4-83B4EC4BD127}  121250  1995-05-19 00:00   
 7       {F7A5CAE4-BDD3-4911-897C-8AC393B97281}   43000  1995-12-01 00:00   
 8       {DED4E551-6B96-40FB-98E9-8AC3B9071356}  128500  1995-03-01 00:00   
 9       {84CE6A8C-3D0F-47F8-8442-8E62807A726D}   33000  1995-11-03 00:00   
 10      {F5F9C3FD-8684-4D4E-B5B8-8E629F77474C}   77500  1995-05-19 00:00   
 11      {D263B73F-7C3A-4062-A432-83B51327C389}   59995  1995-07-21 00:00   

In [27]:
MONGO_HOST = "167.99.88.178"
MONGO_PORT = 27017
MONGO_DB = "houseguru"
USER = "root"
PASS = "COMP6214CW2"

server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_username=USER,
    ssh_password=PASS,
    remote_bind_address=('0.0.0.0', 27017)
)

server.start()

con = pymongo.MongoClient('0.0.0.0',  server.local_bind_port)
db = con[MONGO_DB]
# db.authenticate(MONGO_USER, MONGO_PASS)
places = db.places
propertylistings = db.propertylistings
print(db)

Database(MongoClient(host=['0.0.0.0:56786'], document_class=dict, tz_aware=False, connect=True), 'houseguru')


In [68]:
def convert(dataframe,name,bistype,long,lat):
    dic = []
    for index, row in dataframe.iterrows():
        item = {
            "name":row[name],
            "type":bistype,
            "geocode":[row[long],row[lat]]
        }
        dic.append(item)
    return dic

In [24]:
def export(jsondoc,dbname,filename):
    file = open(filename+".json","w")
    json.dump(jsondoc,file)
    file.close()
    dbname.insert(jsondoc)

In [85]:
foodRatings.head()

['local', 'houseguru', 'admin']

In [86]:
jsondoc = convert(foodRatings,"BusinessName",'food','Geocode/Longitude','Geocode/Latitude')
export(jsondoc,places,'foodratings')

In [260]:
pharmacies.head()

,OrganisationName,Latitude,Longitude
0,Lords Pharmacy,52.244797,0.405598
1,Sainsbury's Pharmacy,54.895298,-2.945134
2,Weldricks Pharmacy,53.512993,-1.132898
3,Rutland Late Night Pharmacy,52.669994,-0.730139
4,Stockport Pharmacy (Internet Only),53.402050,-2.165045


In [87]:
jsondoc = convert(pharmacies,"OrganisationName",'pharmacy','Longitude','Latitude')
export(jsondoc,places,'pharmacies')

In [263]:
crimes.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,NaN,2017-02,Hampshire Constabulary,Hampshire Constabulary,-1.145148,51.353137,On or near Mount Pleasant,E01022553,Basingstoke and Deane 001A,Anti-social behaviour,NaN,NaN
1,NaN,2017-02,Hampshire Constabulary,Hampshire Constabulary,-1.143597,51.350797,On or near Willow Road,E01022553,Basingstoke and Deane 001A,Anti-social behaviour,NaN,NaN
2,NaN,2017-02,Hampshire Constabulary,Hampshire Constabulary,-1.143597,51.350797,On or near Willow Road,E01022553,Basingstoke and Deane 001A,Anti-social behaviour,NaN,NaN
3,b394fb7affc4a51fada38b4e01575c3861a03ebf5fd695...,2017-02,Hampshire Constabulary,Hampshire Constabulary,-1.146835,51.349633,On or near Sarisbury Close,E01022553,Basingstoke and Deane 001A,Public order,Unable to prosecute suspect,NaN
4,199e5bcf281894eb4d6830777cfe0eb93733375dcdf9f4...,2017-02,Hampshire Constabulary,Hampshire Constabulary,-1.147727,51.353443,On or near The Beeches,E01022553,Basingstoke and Deane 001A,Violence and sexual offences,Unable to prosecute suspect,NaN


In [89]:
dic = []
for index, row in crimes.iterrows():
    item = {
        "name":row['Crime type'],
        "type":'crime',
        'date':row['Month'],
        "geocode":[row['Longitude'],row['Latitude']]
    }
    dic.append(item)
export(dic,places,'crimes')

In [90]:
for doc in places.find({"geocode":float('nan')}):
    places.remove(doc)

In [91]:
places.create_index([("geocode",GEO2D)])

'geocode_2d'

In [92]:
query = {"geocode": SON([("$near", [2, 5]), ("$maxDistance", 100)])}
for doc in db.places.find(query).limit(30):
    pprint.pprint(doc)

NameError: name 'pprint' is not defined

# PropertyListing
Fake data

In [32]:
properties

,lat,long,postcode,price
0,50.94141,-1.38462,SO16 3JL,300000
1,50.94335,-1.38438,SO16 3DZ,32000
2,50.94185,-1.38716,SO16 3JH,450000
3,50.93558,-1.38749,SO17 3SW,500000
4,50.93976,-1.39417,SO16 3HA,620000
5,50.93091,-1.38636,SO17 3AG,120000
6,50.93300,-1.39971,SO17 1PX,220000
7,50.93943,-1.39824,SO16 3GN,530000
8,50.94127,-1.37851,SO16 2NQ,236000
9,50.94316,-1.38188,SO16 3DX,890000


In [33]:
dic = []
for index, row in properties.iterrows():
    item = {
        "postcode":row['postcode'],
        "price":row['price'],
        "geocode":[row['long'],row['lat']]
    }
    dic.append(item)
export(dic,propertylisting,'propertylisting')

In [ ]:
server.stop()